In [176]:
!pip3 install pyTigerGraph

In [177]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [178]:
cd '/content/drive/MyDrive/BERN2_TigerGraph_BioMedical_KnowledgeGraph/bioNLP 2'

/content/drive/MyDrive/BERN2_TigerGraph_BioMedical_KnowledgeGraph/bioNLP 2


In [179]:
import pyTigerGraph as tg
import json
import pandas as pd

hostName = "https://mvie.i.tgcloud.io"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


### Clear the Whole Graph
DANGER ZONE

In [180]:
# conn.gsql('''
# USE GLOBAL
# DROP ALL
# ''')

### Create Schema

In [181]:
conn.gsql('''
CREATE VERTEX Sentence(PRIMARY_ID id STRING, text STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Author(PRIMARY_ID id STRING, name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Entity(PRIMARY_ID id STRING, name STRING, other_ids STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Research_Paper(PRIMARY_ID id STRING, title STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Type(PRIMARY_ID type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE DIRECTED EDGE relation(FROM Entity, TO Type) WITH REVERSE_EDGE="reverse_relation"
CREATE DIRECTED EDGE has_entity(FROM Type, TO Entity) WITH REVERSE_EDGE="reverse_has_entity"
CREATE DIRECTED EDGE has_bio_entity(FROM Sentence, TO Entity) WITH REVERSE_EDGE="reverse_has_bio_entity"
CREATE DIRECTED EDGE contains(FROM Research_Paper, TO Sentence) WITH REVERSE_EDGE="reverse_contains"
CREATE DIRECTED EDGE authored(FROM Author, TO Research_Paper) WITH REVERSE_EDGE="reverse_authored"


''')

'Successfully created vertex types: [Sentence].\nSuccessfully created vertex types: [Author].\nSuccessfully created vertex types: [Entity].\nSuccessfully created vertex types: [Research_Paper].\nSuccessfully created vertex types: [Type].\nSuccessfully created edge types: [relation].\nSuccessfully created reverse edge types: [reverse_relation].\nSuccessfully created edge types: [has_entity].\nSuccessfully created reverse edge types: [reverse_has_entity].\nSuccessfully created edge types: [has_bio_entity].\nSuccessfully created reverse edge types: [reverse_has_bio_entity].\nSuccessfully created edge types: [contains].\nSuccessfully created reverse edge types: [reverse_contains].\nSuccessfully created edge types: [authored].\nSuccessfully created reverse edge types: [reverse_authored].'

### Add Index

In [182]:
conn.gsql('''
CREATE GLOBAL SCHEMA_CHANGE JOB global_schema_change_303694980 {
      ALTER VERTEX Sentence ADD INDEX text_q0sexm9h0od ON (text);
      ALTER VERTEX Author ADD INDEX name_b9v9xwpkoxe ON (name);
      ALTER VERTEX Entity ADD INDEX name_mbd9zqa5dgf ON (name);
      ALTER VERTEX Article ADD INDEX title_4x42wkkn972 ON (title);
      ALTER VERTEX Relation ADD INDEX type_q0v48u0xfqm ON (type);
    }

CREATE GLOBAL SCHEMA_CHANGE JOB global_schema_change {
      ALTER VERTEX Sentence ADD INDEX text_q0sexm9h0od ON (text);
      ALTER VERTEX Author ADD INDEX name_b9v9xwpkoxe ON (name);
      ALTER VERTEX Entity ADD INDEX name_mbd9zqa5dgf ON (name);
      ALTER VERTEX Article ADD INDEX title_4x42wkkn972 ON (title);
      ALTER VERTEX Relation ADD INDEX type_q0v48u0xfqm ON (type);
    }
''')

'Semantic Check Fails: The vertex "Article" does not exist in the graph!\nFailed to create global schema change jobs: [global_schema_change_303694980].'

### Create Graph, Load JOBS

In [183]:
conn.gsql('''
      

CREATE GRAPH bio(Sentence, Author, Entity, Research_Paper, Type, relation, has_entity, has_bio_entity, contains, authored)
set exit_on_error = "false"
CREATE LOADING JOB load_job_article_csv_1647329010113 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Research_Paper VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE LOADING JOB load_job_article_sentences_csv_1647329019446 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE contains VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE LOADING JOB load_job_authWROTE_csv_1647329024269 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE authored VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE LOADING JOB load_job_author_csv_1647329032144 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Author VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE LOADING JOB load_job_entity_csv_1647329040312 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Entity VALUES($1, $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE LOADING JOB load_job_rel_type_csv_1647329053478 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Type VALUES($2) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE LOADING JOB load_job_relationship_csv_1647329061926 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE has_bio_entity VALUES($4, $1) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE has_bio_entity VALUES($4, $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE has_entity VALUES($3, $1) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE relation VALUES($2, $3) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE LOADING JOB load_job_sentences_csv_1647329071260 FOR GRAPH bio {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Sentence VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }

set exit_on_error = "true"
set exit_on_error = "false"
CREATE QUERY get_disease_relation(/* Parameters here */) FOR GRAPH bio { 
  /* Write query logic here */ 
  ListAccum<EDGE> @relation;

  start = {Entity.*};

  entity_typ = SELECT s FROM start:s -(relation:r)- :t
               ACCUM s.@relation+=r;

  PRINT entity_typ;
}
set exit_on_error = "true"

''')

'The graph bio is created.\nSuccessfully created loading jobs: [load_job_article_csv_1647329010113].\nSuccessfully created loading jobs: [load_job_article_sentences_csv_1647329019446].\nSuccessfully created loading jobs: [load_job_authWROTE_csv_1647329024269].\nSuccessfully created loading jobs: [load_job_author_csv_1647329032144].\nSuccessfully created loading jobs: [load_job_entity_csv_1647329040312].\nSuccessfully created loading jobs: [load_job_rel_type_csv_1647329053478].\nSuccessfully created loading jobs: [load_job_relationship_csv_1647329061926].\nSuccessfully created loading jobs: [load_job_sentences_csv_1647329071260].\nSuccessfully created queries: [get_disease_relation].'

## Fetch Access Token

In [184]:
graphName = "bio"
conn.graphname = graphName
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)

### Peek into Data to load

In [185]:
df = pd.read_csv('article.csv')
print(df.head(0))
print('######')
df = pd.read_csv('author.csv')
print(df.head(0))
print('######')
df = pd.read_csv('entity.csv')
print(df.head(0))
print('######')
df = pd.read_csv('sentences.csv')
print(df.head(0))
print('######')
df = pd.read_csv('relationship.csv')
print(df.head(0))
print('######')
df = pd.read_csv('authWROTE.csv')
print(df.head(0))
print('######')
df = pd.read_csv('rel_type.csv')
print(df.head(0))
print('######')
df = pd.read_csv('article_sentences.csv')
print(df.head(5))
print('######')

Empty DataFrame
Columns: [Unnamed: 0, id, text]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, id, name]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, id, name, other_ids, type]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, id, text]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, head, tail, type, source]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, sid, did]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, id, type]
Index: []
######
   Unnamed: 0  sid                                                did
0           0    1  e8483fccea2f1e3dfd944a7d82a8406a507e36cbe2369b...
1           1    1  52903d2c573db7e9041a5931c67448a76564b4f05fb380...
2           2    1  1c30070ef4f992ed669f153500040a94c3f2100bf746e8...
3           3    1  fbe3294145c21146ef517e075956cdda59df4c0bea5b3a...
4           4    1  9798e7ec4a5c66c1e31a18578628872c7f29e7bfb18f9a...
######


### Load data

In [186]:
article_file = "article.csv"
res = conn.uploadFile(article_file, fileTag='MyDataSource', jobName='load_job_article_csv_1647329010113')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Research_Paper",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [187]:
author_file = "author.csv"
res = conn.uploadFile(author_file, fileTag='MyDataSource', jobName='load_job_author_csv_1647329032144')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Author",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [188]:
entity_file = "entity.csv"
res = conn.uploadFile(entity_file, fileTag='MyDataSource', jobName='load_job_entity_csv_1647329040312')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 107,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Entity",
          "validObject": 107,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [189]:
sentences_file = "sentences.csv"
res = conn.uploadFile(sentences_file, fileTag='MyDataSource', jobName='load_job_sentences_csv_1647329071260')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 101,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Sentence",
          "validObject": 101,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [190]:
rel_file = "relationship.csv"
res = conn.uploadFile(rel_file, fileTag='MyDataSource', jobName='load_job_relationship_csv_1647329061926')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 13,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "relation",
          "validObject": 13,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "has_entity",
          "validObject": 13,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "has_bio_entity",
          "validObject": 13,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0

In [191]:
aw_file = "authWROTE.csv"
res = conn.uploadFile(aw_file, fileTag='MyDataSource', jobName='load_job_authWROTE_csv_1647329024269')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "authored",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [192]:
df = pd.read_csv("article_sentences.csv")
df

,Unnamed: 0,sid,did
0,0,1,e8483fccea2f1e3dfd944a7d82a8406a507e36cbe2369b...
1,1,1,52903d2c573db7e9041a5931c67448a76564b4f05fb380...
2,2,1,1c30070ef4f992ed669f153500040a94c3f2100bf746e8...
3,3,1,fbe3294145c21146ef517e075956cdda59df4c0bea5b3a...
4,4,1,9798e7ec4a5c66c1e31a18578628872c7f29e7bfb18f9a...
...,...,...,...
95,95,1,b999a94933ca5a63afb339095f089432088ea01e7013d0...
96,96,1,4dc9312a2020c9eee63ef48673b68449608cfb3609d17a...
97,97,1,e9ce6055ec3cf7fb85d32664f1a57aa065ab4d3b02cfa2...
98,98,1,eca418db9658ca1ee0e73299683285918cce6a346bb875...


In [193]:
as_file = "article_sentences.csv"
res = conn.uploadFile(as_file, fileTag='MyDataSource', jobName='load_job_article_sentences_csv_1647329019446')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 101,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "contains",
          "validObject": 101,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [194]:
rel_file = "rel_type.csv"
res = conn.uploadFile(aw_file, fileTag='MyDataSource', jobName='load_job_rel_type_csv_1647329053478')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Type",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]
